# Infusion Optimization


In [ ]:
#import relevant modules
from scripts.preprocessing import *
from scripts.train_main import *
from scripts.infusion import *
import shutil
import os

In [ ]:
#choose infusion splits
infusion_splits = [0.05,0.10,0.15,0.20]

#choose infusion images
infusion_source = 'CC_pig'
#choose base FiN dataset
FiN_dataset = 'FiN_pig'
#choose class
chosen_classes = {'pig'}
#use negative samples in training(True) or not(False)
use_negatives = True
negative_folders = ['negatives']

In [ ]:
#trap dataset/annotations used for infusion
infusion_dataset = './datasets/'+infusion_source+'/'
infusion_annotations = './annotations/'+infusion_source+'/'
training_dataset = './datasets/'+FiN_dataset+'/'

all_infusion_images = []
for each_image in os.listdir(infusion_dataset):
    all_infusion_images.append(each_image)
print("Number of available infusion images: ",len(all_infusion_images))
infusion_images = set()

In [ ]:
#train a model for each infusion percentile
for infusion_split in infusion_splits:
    print("------------------------------------------------------------------------------------")
    separate_infusion(infusion_split, all_infusion_images, infusion_dataset, infusion_annotations, training_dataset,infusion_images)
    chosen_datasets = ['infusion'] 
    chosen_datasets.append(FiN_dataset)
    generate_training_scripts(chosen_datasets, chosen_classes, use_negatives, negative_folders) 
    print("Commencing training...")
    
    #choose training parameters
    pretrained_model = 'pretrained_COCO.h5'
    batch_size = 8
    num_epochs = 30
    #prepare training input
    PRETRAINED_MODEL, BATCH_SIZE, STEPS, EPOCHS = set_training_parameters(pretrained_model, batch_size, num_epochs)
    #default
    SNAPSHOTS = './snapshots/'
    TRAINING = 'training.csv'
    VALIDATION = 'validation.csv'
    CLASSES = 'classes.csv'
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    
    !python3 /homes/une/ashepley/keras-retinanet/keras_retinanet/bin/train.py --freeze-backbone --snapshot-path {SNAPSHOTS} --weights {PRETRAINED_MODEL} --random-transform --batch-size {BATCH_SIZE} --steps {STEPS} --epochs {EPOCHS} csv {TRAINING} {CLASSES}  --val-annotations {VALIDATION}
    
    #rename the training, validation and snapshots files/dir to preserve them
    os.rename('./snapshots', './snapshots'+str(infusion_split)) 
    os.rename('./training.csv', './training'+str(infusion_split)+'.csv')
    os.rename('./validation.csv', './validation'+str(infusion_split)+'.csv')